## <font color = 'green' id = "section" > Projet_7: Implémenter un modèle de scoring</font>

### <font color = 'green' id = "section" > Notebook 1: Dossier Code

# <font color="blue">Sommaire</font>
1. [Importation des librairies](#section_1)    
2. [Lecture du Dataset](#section_2)     
3. [Aperçu des données](#section_3)         
4. [Affichages des informations du Dataset](#section_4)        


# <font color ='red' id = 'section_1' > 1.Importation des librairies </font>

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.warn('DelftStack')
warnings.warn('Do not show this message')
print("No Warning Shown")

No Warning Shown


In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import ast
import scipy as sp
import time
import pickle
#-------------------------------------------------------------------
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import  train_test_split
from sklearn.pipeline import make_pipeline

#-----------------------------------------------------------------
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.compose import TransformedTargetRegressor

#--------------------------------------------
from tqdm import tqdm
import gc
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import collections
from sklearn.neighbors import NearestNeighbors
#----------------------------------------------------
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

 #-----------------------------------------------------------
from sklearn.dummy import DummyClassifier

#------------------------------------------------------------
from lightgbm import LGBMClassifier

#------------------------------------------------------------
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler

#-----------------------------------------------------------
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

#-----------------------------------------------------------
from xgboost import XGBClassifier

#-----------------------------------------------------------
from sklearn.linear_model import LogisticRegression

#-----------------------------------------------------------
from sklearn.metrics import *

#----------------------------------------------------
import lime
from lime import lime_tabular

#------------------------------------------------------------------
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import dash_table

#---------------------------------------------------------------------------
from operator import itemgetter
from flask import Flask
import plotly.express as px
from textwrap import wrap

In [3]:
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

## Checklist
# Mission:

1. Construire un **modèle de scoring** qui donnera une prédiction sur la probabilité de faillite d'un client de façon automatique.


2. Construire un **dashboard interactif** à destination des gestionnaires de la relation client permettant **d'interpréter les prédictions** faites par le modèle, et d’améliorer la connaissance client des chargés de relation client.


# Objectifs/ Spécifications du dashboard:

**1**. Permettre de **visualiser le score et l’interprétation de ce score** pour chaque client de façon intelligible pour une personne non experte en data science.

**2.** Permettre de **visualiser des informations descriptives** relatives à un client (via un système de filtre).


**3.** Permettre de **comparer les informations descriptives** relatives à un client à l’ensemble des clients ou à un groupe de clients similaires.


# Objectifs/ Dossier:

Un dossier sur un outil de versioning de code doit contenir:


* Le **code de la modélisation** (du prétraitement à la prédiction)


* Le **code générant le dashboard**


* Le **code** permettant de **déployer le modèle sous forme d'API**.


# Objectifs/ note méthodologique :

Une note méthodologique décrivant :

* La méthodologie **d'entraînement du modèle** (2 pages maximum)


* La **fonction coût** métier, **l'algorithme d'optimisation** et la **métrique d'évaluation** (1 page maximum)


* **L’interprétabilité globale** et **locale** du modèle (1 page maximum)


* Les **limites** et les **améliorations possibles** (1 page maximum)


# Informations supplémentaires

* Sélectionner un kernel Kaggle pour nous faciliter la préparation des données nécessaires à l’élaboration du modèle de scoring.

* Analyser ce kernel et l’adapter pour nous assurer qu’il répond aux besoins de notre mission.


On peut  ainsi nous  focaliser sur:


* **L’élaboration du modèle**, 


* Son **optimisation**,


* Sa **compréhension**.

# Description des données

lien vers les données: https://www.kaggle.com/c/home-credit-default-risk/data

![title](home_credit.png)

# <font color="red" id="section_2"> 2. Chargement, lecture  et apercu  des données</font>

In [4]:
path = r'C:\Users\sylla\Desktop\Data Sciences\Projet7_ impémenter un model scoring\archive/'

app_test = pd.read_csv(path + "application_test.csv")

path_enc = r"C:\Users\sylla\Desktop\Data Sciences\Projet7_ impémenter un model scoring\P7_Notebook_1_Dossier_code/"
df_test_encoded_features_engi = pd.read_csv(path_enc + "app_test_encodede_feature_engen.csv")

df_cout =pd.read_csv("df_cout.csv")

In [5]:
df_test_encoded_features_engi.head(2)

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,REG_CITY_NOT_LIVE_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_LAST_PHONE_CHANGE,EXT_SOURCE_2,AMT_GOODS_PRICE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,100002,1.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,3.0,0.0,351000.0,10.0,6.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0.077212,0.498036,202500.0,0.121975,0.060748
1,100003,0.0,1,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,2.0,1.0,1129500.0,3.0,1.0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.065359,0.208736,135000.0,0.132215,0.027598


In [6]:
app_train_test = df_test_encoded_features_engi
app_train_prepared = app_train_test[app_train_test['TARGET'].notna()]
app_train_prepared = app_train_prepared.set_index('SK_ID_CURR')
app_test_prepared = app_train_test[app_train_test['TARGET'].isna()]
app_test_prepared = app_test_prepared.set_index('SK_ID_CURR')

In [7]:
app_test_prepared.dtypes

TARGET                                               float64
CODE_GENDER                                            int64
FLAG_OWN_CAR                                           int64
FLAG_OWN_REALTY                                        int64
CNT_CHILDREN                                           int64
REG_CITY_NOT_LIVE_CITY                                 int64
REG_REGION_NOT_LIVE_REGION                             int64
REG_REGION_NOT_WORK_REGION                             int64
LIVE_REGION_NOT_WORK_REGION                            int64
REG_CITY_NOT_WORK_CITY                                 int64
LIVE_CITY_NOT_WORK_CITY                                int64
EXT_SOURCE_1                                         float64
EXT_SOURCE_3                                         float64
AMT_REQ_CREDIT_BUREAU_YEAR                           float64
DAYS_LAST_PHONE_CHANGE                               float64
EXT_SOURCE_2                                         float64
AMT_GOODS_PRICE         

In [8]:
app_test.head(4)

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0505,NaN,NaN,0.0672,0.0612,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0526,NaN,NaN,0.0666,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.125,NaN,NaN,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,NaN,0.699787,0.610991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,Working,Secondary / secondary special,Married

In [9]:
df_test_encoded_features_engi.head(2)

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,REG_CITY_NOT_LIVE_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_LAST_PHONE_CHANGE,EXT_SOURCE_2,AMT_GOODS_PRICE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,100002,1.0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,3.0,0.0,351000.0,10.0,6.0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0.077212,0.498036,202500.0,0.121975,0.060748
1,100003,0.0,1,0,1,0,0,0,0,0,0,0,0.0,0.0,0.0,2.0,1.0,1129500.0,3.0,1.0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.065359,0.208736,135000.0,0.132215,0.027598


In [10]:
df_test_encoded_features_engi = df_test_encoded_features_engi[df_test_encoded_features_engi["TARGET"].isna()]

In [11]:
df_test_encoded_features_engi.head(2)

,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,REG_CITY_NOT_LIVE_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_LAST_PHONE_CHANGE,EXT_SOURCE_2,AMT_GOODS_PRICE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
307506,100001,NaN,1,0,0,0,0,0,0,0,0,0,1.0,0.0,2.0,5.0,1.0,810000.0,17.0,6.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.626436,0.237342,67500.0,0.256296,0.060830
307507,100005,NaN,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,3.0,1.0,373500.0,28.0,1.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0.141538,0.444409,24750.0,0.223414,0.099287


In [12]:
df_test_encoded_features_engi = df_test_encoded_features_engi.set_index("SK_ID_CURR")

In [13]:
df_test_encoded_features_engi.head(3)

,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,REG_CITY_NOT_LIVE_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_LAST_PHONE_CHANGE,EXT_SOURCE_2,AMT_GOODS_PRICE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,NaN,1,0,0,0,0,0,0,0,0,0,1.0,0.0,2.0,5.0,1.0,810000.0,17.0,6.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.626436,0.237342,67500.0,0.256296,0.060830
100005,NaN,0,0,0,0,0,0,0,0,0,0,1.0,1.0,1.0,3.0,1.0,373500.0,28.0,1.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0.141538,0.444409,24750.0,0.223414,0.099287
100013,NaN,0,1,0,0,0,0,0,0,0,0,1.0,0.0,2.0,3.0,1.0,180000.0,14.0,6.0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0.072911,0.305308,202500.0,0.070222,0.021439


# <font color="red" id="section_3"> 3. Affichage des  caracteristiques du jeu de données

### Information

In [14]:
app_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48744 entries, 0 to 48743
Columns: 121 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(40), object(16)
memory usage: 45.0+ MB


In [15]:
df_test_encoded_features_engi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48739 entries, 100001 to 456202
Data columns (total 51 columns):
 #   Column                                             Non-Null Count  Dtype  
---  ------                                             --------------  -----  
 0   TARGET                                             0 non-null      float64
 1   CODE_GENDER                                        48739 non-null  int64  
 2   FLAG_OWN_CAR                                       48739 non-null  int64  
 3   FLAG_OWN_REALTY                                    48739 non-null  int64  
 4   CNT_CHILDREN                                       48739 non-null  int64  
 5   REG_CITY_NOT_LIVE_CITY                             48739 non-null  int64  
 6   REG_REGION_NOT_LIVE_REGION                         48739 non-null  int64  
 7   REG_REGION_NOT_WORK_REGION                         48739 non-null  int64  
 8   LIVE_REGION_NOT_WORK_REGION                        48739 non-null  int64  
 9   

### Duplicité

In [16]:
df_test_encoded_features_engi.duplicated().sum()

0

### Types de variables

In [17]:
df_test_encoded_features_engi.dtypes.value_counts()

int64      37
float64    14
dtype: int64

### Valeurs manquantes

In [18]:
df_test_encoded_features_engi.isna().sum()

TARGET                                               48739
CODE_GENDER                                              0
FLAG_OWN_CAR                                             0
FLAG_OWN_REALTY                                          0
CNT_CHILDREN                                             0
REG_CITY_NOT_LIVE_CITY                                   0
REG_REGION_NOT_LIVE_REGION                               0
REG_REGION_NOT_WORK_REGION                               0
LIVE_REGION_NOT_WORK_REGION                              0
REG_CITY_NOT_WORK_CITY                                   0
LIVE_CITY_NOT_WORK_CITY                                  0
EXT_SOURCE_1                                             0
EXT_SOURCE_3                                             0
AMT_REQ_CREDIT_BUREAU_YEAR                               0
DAYS_LAST_PHONE_CHANGE                                   0
EXT_SOURCE_2                                             0
AMT_GOODS_PRICE                                         

### Valeurs uniques

for col in data_model.columns:
    print("Nombre de  valeurs uniques de", col, "est : ", data_model[col].nunique())
    print("-"*70)

In [19]:
#del df_test_encoded_features_engi["SK_ID_CURR"]

# <font color="red" id="section_4"> 4. Désérialisation 

In [20]:
#lecture du model choisi
pipe_model =pickle.load(open("Model_choice.md", "rb"))

In [21]:
pipe_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['CODE_GENDER',
                                                   'FLAG_OWN_CAR',
                                                   'FLAG_OWN_REALTY',
                                                   'CNT_CHILDREN',
                                                   'REG_CITY_NOT_LIVE_CITY',
                                                   'REG_REGION_NOT_LIVE_REGION',
                                                   'REG_REGION_NOT_WORK_REGION',
                                                   'LIVE_REGION_NOT_WORK_REGION',
                                                   'REG_CITY_NOT_WORK_CITY',
                                                   'LIVE_CITY_NOT_WORK_CITY',

In [22]:
#lecture de la matice de confusiion optimal
conf_mx =pickle.load(open("conf_mx_opt", "rb"))

In [23]:
conf_mx

array([[17312,  8132],
       [ 1115,  1117]], dtype=int64)

# <font color="red" id="section_5"> 5. Approche de mise en oeuvre de dashboad
##  <font color="red" id="section_5_1"> 5.1 Transformation de données test

In [34]:
x_test_transformed = pd.DataFrame(pipe_model[0].transform(df_test_encoded_features_engi.drop(columns=["TARGET"])),
                                  columns=df_test_encoded_features_engi.drop(columns=["TARGET"]).columns,
                                  index=df_test_encoded_features_engi.index)

In [35]:
x_test_transformed.to_csv("x_test_transformed.csv", index = True)

In [36]:
x_test_transformed =  pd.read_csv("x_test_transformed.csv")

In [37]:
x_test_transformed.head(2)

,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,REG_CITY_NOT_LIVE_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_LAST_PHONE_CHANGE,EXT_SOURCE_2,AMT_GOODS_PRICE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
0,100001,0.715789,-0.715467,-0.665612,-0.573224,-0.290454,-0.123233,-0.229386,-0.204122,-0.548531,-0.46816,0.93575,-1.091919,0.044389,0.970852,0.789977,0.738422,0.348215,-0.528058,0.321429,-0.321429,-0.006816,-0.5492,-0.003935,-0.468849,-0.27691,-0.010412,-0.006816,0.967272,-0.325052,0.752968,-0.263179,-0.417494,-0.003935,-0.236662,-0.025205,1.757085,-0.187626,-0.114385,-1.556326,-0.058201,0.356288,-0.193689,-0.093614,-0.126368,-0.226265,2.890146,-0.472670,-0.341924,0.328304,-0.161010
1,100005,-1.397060,-0.715467,-0.665612,-0.573224,-0.290454,-0.123233,-0.229386,-0.204122,-0.548531,-0.46816,0.93575,0.915819,-0.527416,0.007230,0.789977,-0.445750,1.488405,-1.734381,0.321429,-0.321429,-0.006816,-0.5492,-0.003935,-0.468849,-0.27691,-0.010412,-0.006816,0.967272,-0.325052,0.752968,-0.263179,-0.417494,-0.003935,-0.236662,-0.025205,-0.569124,-0.187626,-0.114385,0.642539,-0.058201,0.356288,-0.193689,-0.093614,-0.126368,-0.226265,-0.172307,0.136606,-0.903045,0.120513,0.313065


In [38]:
x_test_transformed  = x_test_transformed.set_index("SK_ID_CURR")

In [39]:
x_test_transformed.head(2)

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,REG_CITY_NOT_LIVE_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_LAST_PHONE_CHANGE,EXT_SOURCE_2,AMT_GOODS_PRICE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,0.715789,-0.715467,-0.665612,-0.573224,-0.290454,-0.123233,-0.229386,-0.204122,-0.548531,-0.46816,0.93575,-1.091919,0.044389,0.970852,0.789977,0.738422,0.348215,-0.528058,0.321429,-0.321429,-0.006816,-0.5492,-0.003935,-0.468849,-0.27691,-0.010412,-0.006816,0.967272,-0.325052,0.752968,-0.263179,-0.417494,-0.003935,-0.236662,-0.025205,1.757085,-0.187626,-0.114385,-1.556326,-0.058201,0.356288,-0.193689,-0.093614,-0.126368,-0.226265,2.890146,-0.472670,-0.341924,0.328304,-0.161010
100005,-1.397060,-0.715467,-0.665612,-0.573224,-0.290454,-0.123233,-0.229386,-0.204122,-0.548531,-0.46816,0.93575,0.915819,-0.527416,0.007230,0.789977,-0.445750,1.488405,-1.734381,0.321429,-0.321429,-0.006816,-0.5492,-0.003935,-0.468849,-0.27691,-0.010412,-0.006816,0.967272,-0.325052,0.752968,-0.263179,-0.417494,-0.003935,-0.236662,-0.025205,-0.569124,-0.187626,-0.114385,0.642539,-0.058201,0.356288,-0.193689,-0.093614,-0.126368,-0.226265,-0.172307,0.136606,-0.903045,0.120513,0.313065


In [ ]:
# Interprétabilité du modèle
explainer = lime_tabular.LimeTabularExplainer(x_test_transformed,
                             feature_names=x_test_transformed.columns,
                             class_names=["O", "1"],
                             discretize_continuous=False)

## <font color="red" id="section_5_2">  5.2 Definition  des fonctions utiles pour le dashboard

In [ ]:
x_test_transformed.head(2)

In [ ]:
pipe_model[3].predict_proba(np.array(x_test_transformed.loc[100001]).reshape(1, -1)).flatten()

In [ ]:
def upadat_proba_bar(id_client):    
    values = pipe_model[3].predict_proba(np.array(x_test_transformed.loc[id_client]).reshape(1, -1)).flatten()
    # Retourne le bar plot mis à jour pour l'id client
    df = ['0', "1"]
    fig_proba = px.bar(x = df, y= values,
     color=['Non faillite', 'Faillite'],
            labels=dict(x='Situation', y='Score proba'),
            title='Information sur le client', text_auto=True)
    fig_proba.update_layout(autosize=False,
        width=500,
        height=500)
    return fig_proba

In [ ]:
upadat_proba_bar(100001)

In [ ]:
def coef_importan():
    import plotly.express as px

    coef_importances_values = pipe_model.named_steps['classifier'].coef_.flatten()
    colors = ['Positive' if c > 0 else 'Negative' for c in coef_importances_values]
    fig = px.bar(
        x=x_test_transformed.columns, y=coef_importances_values, color=colors,
        color_discrete_sequence=['red', 'blue'],
        labels=dict(x='Feature', y='coefficient logistic'),
        title='Coeficient importances'

    )
    fig.update_layout(autosize=False,
        width=980,
        height=800)
    
    fig.update_xaxes(categoryorder = "total ascending")
    return fig

In [ ]:
def coef_importances(pipe_model) :
     
    coef = pipe_model[3].coef_.flatten()
    indices =[]
    values = []
    for val, ind in sorted(zip(pipe_model[3].coef_.flatten(),
                               x_test_transformed.columns), reverse=True):
        indices.append(ind)
        values.append(val)
        
    data = pd.DataFrame(values, columns=["values"], index=indices)
    data["positive"] = data["values"]>0
    del indices, values

    traces = [go.Bar(x=data["values"], y=data.index,
                    orientation='h',
                    marker_color=list(data.positive.map({True: 'red', False: 'blue'}).values))]
     
    return {
        'data': traces,
        
        'layout': go.Layout(margin=dict(l=300, r=0, t=30, b=100))
           }

In [ ]:
coef_importan()

In [ ]:
def infos_client_lime(id_client) :
     
    exp_client = explainer.explain_instance(x_test_transformed.loc[id_client],
                                 pipe_model[3].predict_proba)
    
    indices, values = [], []
    

    for ind, val in sorted(exp_client.as_list(), key=itemgetter(1)):
        indices.append(ind)
        values.append(val)
    
    # Retourne le barplot correspondant aux 'feature importances'
    # du client dont l'id est séléctionné sur le dashboard
    colors = ['Non solvable' if c > 0 else 'Solvable' for c in values]
    fig_lime = px.bar(
        x=indices, y=values, color=colors,
        color_discrete_sequence=['red', 'blue'],
        labels=dict(x='Feature', y='coefficient Lime'),
        title='Information locale sur le client'

    )
    fig_lime.update_layout(autosize=False,
        width=970,
        height=700)

    
    fig_lime.update_xaxes(categoryorder = "total ascending")
    return fig_lime 

In [ ]:
infos_client_lime(100001)

In [ ]:
def plot_mat_conf(conf_mx):
    
    labels = ["False", "True"]
    
    annotations = go.Annotations()
    for n in range(conf_mx.shape[0]):
        for m in range(conf_mx.shape[1]):
            annotations.append(go.Annotation(text=str(conf_mx[n][m]), x=labels[m], y=labels[n],
                                             showarrow=False))
            
            
    colorscale=[[0.0, 'rgb(255, 255, 153)'], [.2, 'rgb(255, 255, 203)'],
            [.4, 'rgb(153, 255, 204)'], [.6, 'rgb(179, 217, 255)'],
            [.8, 'rgb(240, 179, 255)'],[1.0, 'rgb(255, 77, 148)']]

    trace = go.Heatmap(x=labels,
                       y=labels,
                       z=conf_mx,
                       colorscale=colorscale,
                       showscale=True)

    fig_confusion = go.Figure(data=go.Data([trace]))
    
    fig_confusion['layout'].update(
        annotations=annotations,
        xaxis= dict(title='Classes prédites'), 
        yaxis=dict(title='Classes réelles', dtick=1),
        margin={'b': 30, 'r': 20, 't': 10},
        width=500,
        height=300,
        autosize=False
    )
    
    return fig_confusion # Retourne la figure crée

In [ ]:
plot_mat_conf(conf_mx)

##  <font color="red" id="section_5_3"> 5.3 Premiers dashbords avec uniquement les features importances unique
###  <font color="red" > Etape 1: Importation librairies
    
* dash: la librairie Dash, qui comprend :

    - **Dash**: classe Dash
    
    - **html(module Dash HTML Components)**: pour construire la mise en page, qui contient des composants pour chaque balise HTML, comme le titre H1.
    
    - **dcc(module Dash Core Components)** : pour construire la mise en page, qui contient divers composants de niveau supérieur tels que la liste déroulante et le graphique.
    
    - **Input, Output**: pour définir les fonctions de rappel;

In [ ]:
from dash import Dash, html, dcc, Input, Output,dash_table

###  <font color="red" > Etape 2: Construction de l'application dash
    
Chaque application Dash comporte deux parties principales :

* **Mise en page** : détermine à quoi ressemble l'application Dash

* **Fonctions de rappel (callback)**: la fonction qui relie les composants Dash et définit leurs fonctionnalités interactives

Réalisons une visualisation de données interactive à l'aide de Dash.

In [ ]:
app =  Dash()

###  <font color="red" > Etape 3:  Construire la disposition  et  l'interactivité du tableau de bord et 
 
### Disposition
    
Le processus de création d'applications commence toujours à partir de la mise en page. Donc, d'abord, nous devons concevoir l'apparence du tableau de bord.

La mise en page a la structure d'une arborescence de composants. Et nous utilisons le mot-clé **layout du app** pour spécifier sa mise en page. Ensuite, à l'aide des deux modules : **html** et **dcc**, nous pouvons afficher trois composants sur notre tableau de bord, de haut en bas :

    
* **Un titre H1 ( html.H1)** comme titre du tableau de bord. Nous spécifions que sa propriété enfants est le texte **Tableau de bord de la situtation d'un client**.

    
* Un **menu déroulant** basé sur l'identifiant du client.
    

* Un **graphe** ( dcc.Graph) avec l'id : **id_client**.
    
### Interactivité au tableau de bord

Les fonctions de rappel sont des fonctions Python. Ils sont automatiquement appelés par Dash chaque fois que leurs entrées changent. En conséquence, les fonctions s'exécutent et mettent à jour leurs sorties.

Les deux sections principales de la fonction de rappel sont :

* décorateur, qui commence par @app.callback 

* fonction elle-même, qui commence par def


## <font color = 'red' > Étape 4 et fin : Exécuter le tableau de bord
Par défaut, l'application Dash s'exécute sur nos ordinateurs locaux. Pour terminer le script, nous devons ajouter du code pour exécuter le serveur. Nous pouvons ajouter ces deux lignes de code après la fonction de rappel.

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True)

##  <font color="red" id="section_6"> 6. Dashboard global

In [ ]:
x_test_transformed = x_test_transformed.set_index("SK_ID_CURR")
x_test_transformed.head(2)

In [ ]:
µµapp_test = pd.read_csv(path + "application_test.csv")
app_test = app_test.set_index("SK_ID_CURR")

In [ ]:
from dash import Dash, html, dcc, Input, Output, dash_table
import pandas as pd
import numpy as np
import plotly.express as px
import lime
from lime import lime_tabular
import pickle
from operator import itemgetter
from sklearn.neighbors import NearestNeighbors
import plotly.graph_objs as go
import base64
#----------------------------------------------------------------------------------
x_test_transformed = pd.read_csv("x_test_transformed.csv")
x_test_transformed = x_test_transformed.set_index("SK_ID_CURR")
#lecture du model choisi
pipe_model =pickle.load(open("Model_choice.md", "rb"))

#lecture de la matice de confusiion optimal
conf_mx =pickle.load(open("conf_mx_opt", "rb"))

#-----------------------------------------------------------------------------------
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
#---------------------------------------------------------------------------------------

explainer = lime_tabular.LimeTabularExplainer(x_test_transformed,
                             feature_names=x_test_transformed.columns,
                             class_names=["0", "1"],
                             discretize_continuous=False)

#------------------------------------------------------------------------------------------------------

path = r'C:\Users\sylla\Desktop\Data Sciences\Projet7_ impémenter un model scoring\archive/'

app_test = pd.read_csv(path + "application_test.csv")
app_test = app_test.set_index("SK_ID_CURR")
#----------------------------------------------------------------------------------------------------
df_cout = pd.read_csv("df_cout.csv")

#---------------------------------------------------------------------------------------------------

# Calcul des 5 plus proches voisins
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(x_test_transformed)

#-------------------------- liste des vaiables pertinentes----------------------------------
listeInfos = ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','CNT_CHILDREN',
              'AMT_INCOME_TOTAL',
 'AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS',
'DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH',"REG_CITY_NOT_LIVE_CITY",'REG_REGION_NOT_LIVE_REGION',
  'REG_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_WORK_CITY','LIVE_REGION_NOT_WORK_REGION','CNT_FAM_MEMBERS',
              'LIVE_CITY_NOT_WORK_CITY',
'EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_YEAR']

#---------------------------- selection de vaeiable à afficher ------------------------------------
app_test = app_test[listeInfos]
#------------------------------------------------------------------------------------------------------
def plot_mat_conf(conf_mx):
    
    labels = ["False", "True"]
    
    annotations = go.Annotations()
    for n in range(conf_mx.shape[0]):
        for m in range(conf_mx.shape[1]):
            annotations.append(go.Annotation(text=str(conf_mx[n][m]), x=labels[m], y=labels[n],
                                             showarrow=False))
            
            
    colorscale=[[0.0, 'rgb(255, 255, 153)'], [.2, 'rgb(255, 255, 203)'],
            [.4, 'rgb(153, 255, 204)'], [.6, 'rgb(179, 217, 255)'],
            [.8, 'rgb(240, 179, 255)'],[1.0, 'rgb(255, 77, 148)']]

    trace = go.Heatmap(x=labels,
                       y=labels,
                       z=conf_mx,
                       colorscale=colorscale,
                       showscale=True)

    fig_confusion = go.Figure(data=go.Data([trace]))
    
    fig_confusion['layout'].update(
        annotations=annotations,
        xaxis= dict(title='Classes prédites'), 
        yaxis=dict(title='Classes réelles', dtick=1),
        margin={'b': 30, 'r': 20, 't': 10},
        width=600,
        height=450,
        autosize=False
    )
    
    return fig_confusion # Retourne la figure crée
#-------------------------------------------------------------------------------------------------
def coef_importances(pipe_model) :
     
    coef = pipe_model[3].coef_.flatten()
    indices =[]
    values = []
    for val, ind in sorted(zip(pipe_model[3].coef_.flatten(),
                               x_test_transformed.columns), reverse=True):
        indices.append(ind)
        values.append(val)
        
    data = pd.DataFrame(values, columns=["values"], index=indices)
    data["positive"] = data["values"]>0
    del indices, values

    traces = [go.Bar(x=data["values"], y=data.index,
                    orientation='h',
                    marker_color=list(data.positive.map({True: 'red', False: 'blue'}).values))]
     
    return {
        'data': traces,
        
        'layout': go.Layout(margin=dict(l=300, r=0, t=30, b=100))
           }


def fonction_cout():
    x_axis = df_cout["threshold"]
    y_axis_cout = df_cout["cost"]
    min_cout = df_cout['cost'].min()
    fig = go.Figure()
    traces = go.Scatter(
            x= x_axis,
            y= y_axis_cout,
            mode= 'lines+markers',
            y0 =min_cout,
            line={'shape':'spline','smoothing':1},
            name= "Fonction cout")
  

    return {
        'data': [traces],
        'layout': go.Layout(
            title = "Evolution du coût par rapport aux thresholds",
            xaxis= dict(title = "Thresholds"),
            yaxis= dict(title = "Coût"),
          
            hovermode= 'closest',
            legend_orientation= 'h'

        )
    }

#-------------------------------------- table des client similaire-----------------------------------



#-------------------------------------------------------------------------------------
variables = ['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE',
                 'CNT_FAM_MEMBERS' ,'EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3', "AMT_REQ_CREDIT_BUREAU_YEAR" ]

variable_inf = ['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE',
                 'CNT_FAM_MEMBERS' ,'EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3', "AMT_REQ_CREDIT_BUREAU_YEAR" ]


#----------------------------------------------------------------------------------------------------------

image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
#image_path = "C:\Users\sylla\Desktop\Data Sciences\Projet7_ impémenter un model scoring\P7_Notebook_2_Dashbord/logo.png"

#-----------------------------------------------------------------------------------------------------
markdown_text = '''
### Seuil de probabilté

Le seuil de probalilité obtenu après optimisation de la
fonction coût est de: **0.54123**.

En sous de cette  valeur, **demande acceptée**;


Sinon, **refusée**.
'''

#-------------------------------------------------------------------------
                     
################################################################################################################""
########################################################################################################################"
app = Dash(__name__)
# definition de la colone vertebrale du dashboad avzec dcc qui permet de creer des onglers
id_dropdown = dcc.Dropdown( id='id_client',
                           options=x_test_transformed.index,
                            value=x_test_transformed.index[0],
                           multi = True
                          )
## Définition de loyout de app

app.layout = html.Div([
   html.Div( children =[
       
         html.Div([
            #Mettre un titre au dropdown
            html.H4("Choisissez l'identifiant d'un client:"),
            ## deuxieme composante un dcc
            dcc.Dropdown(
                ##donner un id
                id = 'id_client',
                #on choisit l'option qu'est je vais mettre à l'interieur du dropdown (ici les identifiant des clients
                options = x_test_transformed.index,
                #on on peut même donner une valeur par defaut le client à l'index 0
                value = x_test_transformed.index[0]
            )
        ], style = {
             'background-color': 'rgb(200, 200, 200)',
    
            'width': "25%", # 25 % de la page
            "border": '1px solid #eee',
            'padding': '30px 30px 30px 12px',
            'box-shadow': '0 2px 2px #ccc',
             'display': 'inline_block',
            'textAlign': 'left'}),
  
           
                                   ]
             
            
        ), 
#----------------------------------------------------------------------------------------------    
    #On va definir les onglet à farire apparaitre sur le tableau de bord
    html.Div([
        dcc.Tabs(id ='tab', value ='tab-1', # les children sont les enfants de de Tabs 
                children = [
           ###################################################################################################         
               # 1 ier onglet: infos client
                    dcc.Tab(label = 'Infos client', children = [
                         html.Div([html.H3('Infos client') 
                             
                         ], style ={'background': 'blue',
                                          "color": "white", 
                                    'textAlign':'center',
                                'padding':'10px 0px 10px 0px'}),
                        
                        
                        
                        html.Div([
                            dash_table.DataTable(
                                id = "table_infos",style_cell = {'font-family': 'Montserrat'}, 
                    style_data_conditional  =[{
                        
                        'if': {'column_id': 'intitule'},
                        'textAlign': 'left'
                    }] +
                    
                    [{
                        
                        'if': {'row_index': 'odd'}, # si les ligens sont impaire
                        'backgroundColor ': 'rgb(248,248, 248)'
                    }], style_header = { 'backgroundColor ': 'rgb(230,230, 230)',       #l'entête un peu plus foncé
                                        'fontWeight': 'bold'
                        
                    }
                           
                )], style= {'display':'inline-block',
                            'verticalAlign':'top',
                            'width': '45%', 
                            'padding':'40px 0px 0px 10px'} ),
                        
                        
                        html.Div([
                            dcc.Graph(id = 'pie_infos')
                            
                            
                        ], style= {'display':'inline-block', 
                                   'verticalAlign':'top',
                                   'width': '45%',
                                  'padding':'70px 0px 0px 70px',
                                  'box-shadow': '2px 2px 2px  #ccc'}),
           #**********************************************************************************************************             
           #--------------------liste deroulante de toute les variable de la table  client simil----------------------
                     html.Div([  
            
            html.H4("Choisissez une variable:"),
            ## deuxieme composante un dcc
            dcc.Dropdown(
                ##donner un id
                id = 'variable_info',
                #on choisit l'option qu'est je vais mettre à l'interieur du dropdown (ici les identifiant des clients
                options = variable_inf,
                #on on peut même donner une valeur par defaut le client à l'index 0
                value = variable_inf[0])

                     ], style = {'width': "25%", # 25 % de la page
                                            "border": '1px solid #eee',
                                            'padding': '30px 30px 30px 120px',
                                            'box-shadow': '2px 2px 0px #ccc',
                                             'display': 'inline_block',
                                            'verticalAlign': 'top'}),
      #-------------------------------------- Titre -------------------------------------------
                      html.Div([html.H3('Information du client choisi par rapport à la moyenne de la population') 
                             
                         ], style ={'background': 'blue',
                                          "color": "white", 
                                    'textAlign':'center',
                                'padding':'10px 0px 10px 0px'}),
        #___________________________________________ graphe moyenne_________________________________________________  
                        html.Div([
                            dcc.Graph(id = 'bar_moyenne')
                            
                            
                        ], style= {'display':'inline-block', 
                                   'verticalAlign':'center',
                                   'width': '90%',
                                  'padding':'70px 0px 0px 70px',
                                  'box-shadow': '2px 2px 2px  #ccc'}),
                        
                        
                        
                                                                ],
                           ),
        ################################################################################################
                    # 2 ieme onglet: Score de probabilté de faillite
                    dcc.Tab(label = 'Situation du client', children =[
                        
                #****************************************************************************************      
           
                 html.Div([html.H3("Acceptabilité de la demande de prêt du client ")], style ={'background': 'blue',
                                                                                             "color": "white", 'textAlign':'center',
                                                                                           'padding':'10px 0px 10px 0px'}),          
                        
                   html.Div(id='result', children=["Resultat"], style={'background': '',
                                                                       "color": "black",
                                                                      # "height": '50px',
                                                                       'border': '5px dotted red',
                                                                       'padding':'50px 3px 50px 50px'}), 
                        
                     #------------------------------------------------------------------------------   
                        
                         html.Div([
                                        dcc.Markdown(children=markdown_text)
                                    ],style ={'background': '',
                                                "color": "black", 'textAlign':'center',
                                                 'border': '10px solid #ccc', 
                                                  'padding':'10px 0px 10px 0px'} ),
               
        #****************************************************************************************       
                html.Div([html.H3("Eléments justifiant la décision de prêt ou non ")], style ={'background': 'blue',
                                                                                             "color": "white", 'textAlign':'center',
                                                                                    'padding':'10px 0px 10px 0px'}),
                        html.Div([
                    dcc.Graph(id = 'proba_value'),
                            
                   html.P("Position of hline"),
                dcc.Slider(
                id='slider-position', 
                        min=0, max=1, value=0.54123, step=0.54123,
                    marks={0: '0',0.54123:'0.54123', 1: '1'}
                                        ),
                     ], style = {'border': '1px solid #ccc', 
                                #'box-shadow': '0 2px 2px #ccc',
                                 'display': 'inline-block',
                                 'verticalAlign': 'top',
                                 'width': '45%',
                                 'padding': '0px 0px 0px 0px'
                                }),
                        #____________________________________________________________
                        html.Div([
                    dcc.Graph(id = 'graph_lime')
                     ], style = {#'border': '1px solid #ccc', 
                                #'box-shadow': '0 2px 0px #ccc',
                                 'display': 'inline-block',
                                 'verticalAlign': 'top',
                                 'width': '45%',
                                'padding': '50px 0px 0px 30px'
                                }), 
                        #____________________________________________________________client similaires__________________________
                      
             html.Div([html.H3("Tableau des données du client choisi et de ses quatre plus proches voisins ?")], 
                      style ={'background': 'blue',
                               "color": "white", 'textAlign':'center',
                           'padding':'10px 0px 10px 0px'}),
                        
                  html.Div([            
                    dash_table.DataTable(
                                            id='client_similaire',
                                            columns=[  {"name": i, "id": i} for i in app_test.reset_index().columns],
                                            
                                           filter_action='custom',
                                            filter_query='',
                                            fixed_rows={'headers': True, 'data': 0 },
                                           style_cell={'width': '200px'},
                                            style_table={'minWidth': '90%'},
                                            style_data_conditional=[
                                                                    {'if': {'row_index': 'odd'},
                                                                    'backgroundColor': 'rgb(248, 248, 248)' 
                                                                    }],
                                            style_header={
                                                            'backgroundColor': 'rgb(230, 230, 230)',
                                                            'fontWeight': 'bold'
                                                        }, 
                                            virtualization=True,
                                         )


                                  ],style ={'width': '90%', "border": '1px solid #eee',
                            'box-shadow': '0 2px 2px #ccc',
                             'display': 'inline_block',
                            'verticalAlign': 'top',
                            'padding':'60px 30px 60px 30px'
                                           } ),
                          
                        
                                  
                  #--------------------liste deroulante de toute les variable de la table  client simil----------------------
                                #--------------------liste deroulante de toute les variable de la table  client simil----------------------
                     html.Div([  
            
            html.H4("Choisissez une variable:"),
            ## deuxieme composante un dcc
            dcc.Dropdown(
                ##donner un id
                id = 'variable',
                #on choisit l'option qu'est je vais mettre à l'interieur du dropdown (ici les identifiant des clients
                options = variables,
                #on on peut même donner une valeur par defaut le client à l'index 0
                value = variables[0])

                     ], style = {'width': "25%", # 25 % de la page
                                            "border": '1px solid #eee',
                                            'padding': '30px 30px 30px 120px',
                                            'box-shadow': '2px 2px 0px #ccc',
                                             'display': 'inline_block',
                                            'verticalAlign': 'top'}),
      #-------------------------------------- Titre -------------------------------------------
                        
                  html.Div([  
            
                               html.Div([html.H3("Comparaison du client choisi par rapport à ses plus proches voisins")],
                                              style ={'background': 'blue',
                                                        "color": "white", 'textAlign':'center',
                                                        'padding':'10px 0px 10px 0px'}),
#------------------------------------------------------- Grphe 1---------------------------------------------

                                ]),
                        html.Div([
            dcc.Graph(id = 'graph_comp_1')
                    ], style = {'border': '1px solid #ccc', 
                                'box-shadow': '0 2px 2px #ccc',
                                 'display': 'inline-block',
                                 'verticalAlign': 'top',
                                 'width': '90%',
                                 'padding': '50px 0px 0px 50px'} ), 
   #------------------------------------------------------fin graphe ------------------------------------------------------
                        
 #------------------------------------------------------- Grphe 2---------------------------------------------


                        
                                                           ]),

                                 
   
           #*****************************************************************************************         
                            # 3ieme onglet: Grahique des coef importances local

                         dcc.Tab(label="Performance du model", children=[
                             
                                   #------------------------------- Matrice de confusion ------------------------
                                  html.Div([
                                      html.H3("Matrice de confusion et coefficients importances"),
                                            
                                           ], style ={'background': 'blue',
                                                    "color": "white", 'textAlign':'center',
                                                    'padding':'10px 0px 10px 0px'},
                                           ),
                                  html.Div([
                            # Affiche la matrice de confusion obtenue apres optimisation fonction cout
                                            
                                            html.Div([
                                                html.H3("Matrice de confusion"), 
                                                dcc.Graph(id='matrice_conf',
                                                          figure= plot_mat_conf(conf_mx),
                                                         ),
                                                     ], style = {'border': '1px solid #ccc', 
                                                                'box-shadow': '0 2px 2px #ccc',
                                                                 'display': 'inline-block',
                                                                 'verticalAlign': 'top',
                                                                 'width': '45%',
                                                                 'padding': '50px 0px 0px 50px'
                                                                }),
                    #------------------------------- coeficicient importances globales------------------------
                        
                                            html.Div([
                                                html.H3("Coeficient importances"), 
                                                dcc.Graph(id='coef_importance',
                                                          figure= coef_importances(pipe_model),
                                                         ),
                                                     ],    style = {'border': '1px solid #ccc', 
                                                                    'box-shadow': '0 2px 2px #ccc',
                                                                     'display': 'inline-block',
                                                                     'verticalAlign': 'top',
                                                                     'width': '45%',
                                                                     'padding': '50px 0px 0px 50px'
                                                                    }),

                                         ]),

                                                                 ],
                                ),
               
                       
          #********************************************************************************************************************              
                    
                     # 4 ième onglet: Tableau fonction cout
                    dcc.Tab(label = 'Fonction coût', children = [
                                  
        #-------------------------------------------------------petit titre de l'inglet-----------------------
                    html.Div([html.H3('Graphique de la fonction de coût optimisée'),
                             ],
                              style = {'background': 'blue',
                                         "color": "white", 'textAlign':'center',
                                          'padding':'10px 0px 10px 0px'}),       
                           
                        html.Div([
                            dcc.Graph(id = 'cout', 
                                 figure = fonction_cout(),
                                     
                                 ),
                        ], style = {'border': '1px solid #ccc', 
                                      'box-shadow': '2px 2px 2px #ccc',
                                        'display': 'inline-block',
                                        'verticalAlign': 'top',
                                        'width': '80%',
                                            'padding': '50px 0px 0px 100px'})   
                                                                    ],
                           ),
      
                        
                    
         #******************************************************************************************************  
                    #4 iéme onglet : Exploration des données
                    #dcc.Tab(label = 'Exploration des données'),
                    
                ])
                
        
    ])
   
])
#*************************************************************************************************************
#******************************************************    *******************************************************
# Création d'un système de filtre
operators = [['ge ', '>='],
             ['le ', '<='],
             ['lt ', '<'],
             ['gt ', '>'],
             ['ne ', '!='],
             ['eq ', '='],
             ['contains '],
             ['datestartswith ']]
def split_filter_part(filter_part):
    # Permet d'avoir un outil de filtrage des données
    for operator_type in operators:
        for operator in operator_type:
            if operator in filter_part:
                name_part, value_part = filter_part.split(operator, 1)
                name = name_part[name_part.find('{') + 1: name_part.rfind('}')]

                value_part = value_part.strip()
                v0 = value_part[0]
                if (v0 == value_part[-1] and v0 in ("'", '"', '`')):
                    value = value_part[1: -1].replace('\\' + v0, v0)
                else:
                    try:
                        value = float(value_part)
                    except ValueError:
                        value = value_part

                # word operators need spaces after them in the filter string,
                # but we don't want these later
                return name, operator_type[0].strip(), value

    return [None] * 3
#*******************************************       ***************************************************
#**************************************************************************************************************








#--------------------------------------------------------------------------------------
# Definition fonction de rappel callback

#----------------------------------------------------------------------------------------------------
#-------------------------------------------------------------------------------------------------------------

############################ infos client #######################################################""

@app.callback([Output('table_infos', 'data'),Output('table_infos', 'columns')],
              [Input("id_client", 'value')]) #obliger de la mettre entre crocher meme si c"est un seul elemen
def infos_client(id_client):
    infos_list = ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY', 'NAME_INCOME_TYPE',
                  'NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS',"REG_CITY_NOT_LIVE_CITY",'REG_REGION_NOT_LIVE_REGION',
                'REG_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_WORK_CITY','LIVE_REGION_NOT_WORK_REGION','CNT_FAM_MEMBERS',
              'LIVE_CITY_NOT_WORK_CITY', 'AMT_INCOME_TOTAL']
    
    #info_list = [info  for info in col ]
    infos = {'intitule':infos_list,
            'donnee': [app_test[app_test.index==id_client][col].iloc[0] for col in infos_list]
            }
    
    table = pd.DataFrame(infos)
    data = table.to_dict('rows')
    entete = {'id':'intitule', 'name': "Principaux indicateurs"}, {'id': 'donnee',
                                                                                           'name': "Id_client: " + str(id_client)}
    
    return data, entete
    #--------------------------------------- grahe pie client----------------------------------------------------#
@app.callback(Output('pie_infos', 'figure'),
              [Input('id_client', 'value')])
def graph_pie(id_client):
    col_pie = ['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_GOODS_PRICE']
    data_info = [float(app_test[app_test.index ==id_client][col].iloc[0]) for col in col_pie]
    labels =  col_pie
   
    traces = [
        go.Bar(x= labels, y =data_info, text =data_info , textposition='auto')
    ]
    del data_info
    return {
        'data': traces,
        'layout': go.Layout(
           
            title= 'Information client',
            #legend_orientation= 'h'
        )
    }
############################################## acceptabilité de la demande ##################""
@app.callback(
    Output('result', 'children'),
    [Input('id_client', 'value')])

def update_result(id_client):
    proba = pipe_model[3].predict_proba(np.array(x_test_transformed.loc[id_client]).reshape(1, -1)).flatten()
    proba = proba[1]
    if (proba < 0.541237): 
        return "Demande de prêt accéptée !"
    else:
        return "Demande de prêt refusée !"


############################ probabilté de faillite #######################################################""

@app.callback(Output('proba_value', 'figure'),
             [Input('id_client', 'value')])
def update_proba_bar(id_client):    
    proba = pipe_model[3].predict_proba(np.array(x_test_transformed.loc[id_client]).reshape(1, -1)).flatten()
    prod = np.array([proba[1]])
    df = ["Classe"]
    fig =px.bar(x = df, y= prod,
                labels=dict(x = '...= Seuil', y='Score proba'),
               title='Le score de probabilité  pour prise de décision ', text_auto=True)
    fig.add_hline(y=0.8, line_color="black", line_dash="dot", line_width = 0.001)
    fig.add_hline(y=0.54123, line_color="red", line_dash="dot", line_width = 2)
    return fig
    del proba
#****************************************graph de lime**********************************
@app.callback(Output('graph_lime', 'figure'),
             [Input('id_client', 'value')])
def graphe_lime(id_client) :
     
    exp = explainer.explain_instance(x_test_transformed.loc[id_client],
                                 pipe_model[3].predict_proba)
    
    indices, values = [], []
    

    for ind, val in sorted(exp.as_list(), key=itemgetter(1)):
        indices.append(ind)
        values.append(val)
    data = pd.DataFrame(values, columns=["values"], index=indices)
    data["positive"] = data["values"]>0
    del indices, values
    
    # Retourne le barplot correspondant aux 'feature importances'
    # du client dont l'id est séléctionné sur le dashboard
    traces = [go.Bar(
                    x=data["values"],
                    y=data.index,
                    orientation='h',
                    marker_color=list(data.positive.map({True: 'red', False: 'green'}).values)
        )]
    return {
        
        'data': traces,
        
        'layout': go.Layout(
                            margin=dict(l=300, r=0, t=30, b=100),
            title = "Influences des variables(Vert facilitant Non faillite et rouge  faillite)"
        )  
                           
    } 
############################################ client similaire ################################
@app.callback(
    Output('client_similaire', 'data'),
    [Input('client_similaire', "filter_query"),
     Input('id_client', "value")])
def update_table(filter, id_client):
    
    # Déterminer les individus les plus proches du client dont l'id est séléctionné
    indices_similary_clients = nbrs.kneighbors(np.array(x_test_transformed.loc[id_client]).reshape(1, -1))[1].flatten()
     
    filtering_expressions = filter.split(' && ')
    dff = app_test.iloc[indices_similary_clients].reset_index()
    for filter_part in filtering_expressions:
        col_name, operator, filter_value = split_filter_part(filter_part)

        if operator in ('eq', 'ne', 'lt', 'le', 'gt', 'ge'):
            # these operators match pandas series operator method names
            dff = dff.loc[getattr(dff[col_name], operator)(filter_value)]
        elif operator == 'contains':
            dff = dff.loc[dff[col_name].str.contains(filter_value)]
        elif operator == 'datestartswith':
            # this is a simplification of the front-end filtering logic,
            # only works with complete fields in standard format
            dff = dff.loc[dff[col_name].str.startswith(filter_value)]
    
    return dff.to_dict('records')

   ################################### graphe_comp_1 ###################### 
@app.callback(Output('graph_comp_1', 'figure'),
             [Input('id_client', 'value'), Input('variable', 'value')])
def graph_comparaison_1(id_client,variable):
    indices_similary_clients = nbrs.kneighbors(np.array(x_test_transformed.loc[id_client]).reshape(1, -1))[1].flatten()
    df_client_simil = app_test.iloc[indices_similary_clients]
 
    ind_client_vs = df_client_simil.index.to_list()
    
    data_client_princi = df_client_simil[df_client_simil.index == id_client][variable].iloc[0]
    data_client_1 = df_client_simil[df_client_simil.index  == ind_client_vs[1]][variable].iloc[0]
    data_client_2 = df_client_simil[df_client_simil.index  == ind_client_vs[2]][variable].iloc[0]
    data_client_3 = df_client_simil[df_client_simil.index  == ind_client_vs[3]][variable].iloc[0]
    data_client_4 = df_client_simil[df_client_simil.index  == ind_client_vs[4]][variable].iloc[0]

    labels = ['Client choisi', 'Voisin_1', "Voisin_2", "Voisin_3", "Voisin_4"]
    values = [float(data_client_princi), float(data_client_1),float(data_client_2), float(data_client_3),
              float( data_client_4) ]
    
    total = sum(values)

    traces = [
        go.Pie(labels= labels, values= values, texttemplate = "%{label}: %{value:s} <br>(%{percent})",
    textposition = "inside")
    ]

    return {
        'data': traces,
        'layout': go.Layout(
        title= " Information du client choisi comparée <br> à celle de ses 4 + proches voisins  (Total: " + str(total) + ")",
            #legend_orientation= 'h'
        )
    }

##################################################Graphe moyenne #####################################################
@app.callback(Output('bar_moyenne', 'figure'),
             [Input('id_client', 'value'), Input('variable_info', 'value')])
def graph_bar_moyenne(id_client,variable_info):

    
    data_client_princi = app_test[app_test.index == id_client][variable_info].iloc[0]
    data_moy = app_test[variable_info].mean()
 

    labels = ['Client choisi', "Moy_Pop"]
    values = [float(data_client_princi), float(data_moy)]
    
    total = sum(values)

    traces = [
        go.Pie(labels= labels, values= values, hole =0.5, texttemplate = "%{label}: %{value:s} <br>(%{percent})",
    textposition = "inside")
    ]

    return {
        'data': traces,
        'layout': go.Layout(
        title= " Information du client choisi comparée <br> à la moyenne de la population (Total: " + str(total) + ")",
            #legend_orientation= 'h'
        )
    }






server =  app.server  
if __name__ == "__main__":
    app.run_server(port = 8002) 

In [40]:
#x_test_transformed  = x_test_transformed.set_index("SK_ID_CURR")

In [41]:
x_test_transformed.head(2)

,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,REG_CITY_NOT_LIVE_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_LAST_PHONE_CHANGE,EXT_SOURCE_2,AMT_GOODS_PRICE,DAYS_REGISTRATION,DAYS_ID_PUBLISH,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,DAYS_EMPLOYED_PERC,INCOME_CREDIT_PERC,INCOME_PER_PERSON,ANNUITY_INCOME_PERC,PAYMENT_RATE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,0.715789,-0.715467,-0.665612,-0.573224,-0.290454,-0.123233,-0.229386,-0.204122,-0.548531,-0.46816,0.93575,-1.091919,0.044389,0.970852,0.789977,0.738422,0.348215,-0.528058,0.321429,-0.321429,-0.006816,-0.5492,-0.003935,-0.468849,-0.27691,-0.010412,-0.006816,0.967272,-0.325052,0.752968,-0.263179,-0.417494,-0.003935,-0.236662,-0.025205,1.757085,-0.187626,-0.114385,-1.556326,-0.058201,0.356288,-0.193689,-0.093614,-0.126368,-0.226265,2.890146,-0.472670,-0.341924,0.328304,-0.161010
100005,-1.397060,-0.715467,-0.665612,-0.573224,-0.290454,-0.123233,-0.229386,-0.204122,-0.548531,-0.46816,0.93575,0.915819,-0.527416,0.007230,0.789977,-0.445750,1.488405,-1.734381,0.321429,-0.321429,-0.006816,-0.5492,-0.003935,-0.468849,-0.27691,-0.010412,-0.006816,0.967272,-0.325052,0.752968,-0.263179,-0.417494,-0.003935,-0.236662,-0.025205,-0.569124,-0.187626,-0.114385,0.642539,-0.058201,0.356288,-0.193689,-0.093614,-0.126368,-0.226265,-0.172307,0.136606,-0.903045,0.120513,0.313065


In [29]:
x_test_transformed = pd.DataFrame(array)
x_test_transformed.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
0,0.715789,-0.715467,-0.665612,-0.573224,-0.290454,-0.123233,-0.229386,-0.204122,-0.548531,-0.46816,0.93575,-1.091919,0.044389,0.970852,0.789977,0.738422,0.348215,-0.528058,0.321429,-0.321429,-0.006816,-0.5492,-0.003935,-0.468849,-0.27691,-0.010412,-0.006816,0.967272,-0.325052,0.752968,-0.263179,-0.417494,-0.003935,-0.236662,-0.025205,1.757085,-0.187626,-0.114385,-1.556326,-0.058201,0.356288,-0.193689,-0.093614,-0.126368,-0.226265,2.890146,-0.472670,-0.341924,0.328304,-0.161010
1,-1.397060,-0.715467,-0.665612,-0.573224,-0.290454,-0.123233,-0.229386,-0.204122,-0.548531,-0.46816,0.93575,0.915819,-0.527416,0.007230,0.789977,-0.445750,1.488405,-1.734381,0.321429,-0.321429,-0.006816,-0.5492,-0.003935,-0.468849,-0.27691,-0.010412,-0.006816,0.967272,-0.325052,0.752968,-0.263179,-0.417494,-0.003935,-0.236662,-0.025205,-0.569124,-0.187626,-0.114385,0.642539,-0.058201,0.356288,-0.193689,-0.093614,-0.126368,-0.226265,-0.172307,0.136606,-0.903045,0.120513,0.313065


In [45]:
pipe_model[3].predict_proba(np.array(x_test_transformed.loc[100001]).reshape(1, -1))

array([[0.57357325, 0.42642675]])